In [1]:
!pip install transformers datasets

import numpy as np
import tensorflow as tf
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from datasets import load_dataset
from sklearn.model_selection import train_test_split


Defaulting to user installation because normal site-packages is not writeable


In [2]:
dataset = load_dataset("kmyoo/cnn-dailymail-v1-tiny", split="train")

In [3]:
import os
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = 'true'


In [4]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
def tokenize_function(examples):
    inputs = examples['article']
    targets = examples['highlights']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs


In [6]:
from transformers import AutoTokenizer, T5ForConditionalGeneration


In [9]:
from transformers import AutoTokenizer, T5ForConditionalGeneration
from datasets import load_dataset
import torch

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Load dataset
dataset = load_dataset("kmyoo/cnn-dailymail-v1-tiny", split='train')

# Extract texts and labels from the dataset
texts = [example['article'] for example in dataset]
labels = [example['highlights'] for example in dataset]

# Tokenize inputs and labels
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
labels = tokenizer(text_target=labels, padding=True, truncation=True, return_tensors='pt')

# Prepare data for training
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
labels = labels['input_ids']

# Training configuration
num_epochs = 3
batch_size = 8
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

# Training loop (simplified)
model.train()
for epoch in range(num_epochs):
    for start in range(0, len(input_ids), batch_size):
        end = start + batch_size
        batch_input_ids = input_ids[start:end]
        batch_attention_mask = attention_mask[start:end]
        batch_labels = labels[start:end]

        optimizer.zero_grad()
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask, labels=batch_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')


Epoch 1/3, Loss: 6.662642955780029
Epoch 2/3, Loss: 2.7266714572906494
Epoch 3/3, Loss: 1.669614553451538


In [10]:
def tokenize_function(examples):
    inputs = examples['article']
    targets = examples['highlights']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs


In [11]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['article', 'highlights'])
tokenized_dataset.set_format('torch')


In [12]:
train_dataset, eval_dataset = train_test_split(tokenized_dataset, test_size=0.1, random_state=42)



In [13]:
!pip install transformers[torch] accelerate -U


Defaulting to user installation because normal site-packages is not writeable


In [14]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    output_dir='./results',
    num_train_epochs=3,
    save_steps=10_000,
    eval_steps=5_000,
    logging_dir='./logs',
    logging_steps=200,
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

In [25]:
for idx in range(5):
    print(dataset[idx])


{'article': "At least nine people have died in clashes between protesters and police Turkey as demonstrations against ISIS's advance on a town on the Syrian-Turkish border turned violent. Turkey's pro-Kurdish Peoples' Democratic Party had called for citizens to protest the assault on Kobane, where the situation turned 'extremely critical' overnight. Officers were forced to use tear gas and water cannons to disperse protesters in Turkey's predominantly Kurdish eastern and southeastern provinces, as well as\xa0the capital Ankara and in Istanbul, where cars were set on fire and demonstrators threw rocks and fireworks at police. Scroll down for video . Burning streets: Kurdish protesters are pictured in a street in the southeastern city of Diyarbakir as the anti-ISIS demonstrations across Turkey on Tuesday continue into the night . Explosions: Protesters throw fireworks at riot police on the streets of Istanbul on Tuesday evening . Day of violence: Smokes rises from the Gaziosmanpasa distr

In [31]:
for i in range(5):
    try:
        print(dataset[i])
    except KeyError as e:
        print(f"KeyError at index {i}: {e}")


{'article': "At least nine people have died in clashes between protesters and police Turkey as demonstrations against ISIS's advance on a town on the Syrian-Turkish border turned violent. Turkey's pro-Kurdish Peoples' Democratic Party had called for citizens to protest the assault on Kobane, where the situation turned 'extremely critical' overnight. Officers were forced to use tear gas and water cannons to disperse protesters in Turkey's predominantly Kurdish eastern and southeastern provinces, as well as\xa0the capital Ankara and in Istanbul, where cars were set on fire and demonstrators threw rocks and fireworks at police. Scroll down for video . Burning streets: Kurdish protesters are pictured in a street in the southeastern city of Diyarbakir as the anti-ISIS demonstrations across Turkey on Tuesday continue into the night . Explosions: Protesters throw fireworks at riot police on the streets of Istanbul on Tuesday evening . Day of violence: Smokes rises from the Gaziosmanpasa distr